In [5]:
from flask import Flask, render_template, redirect, url_for, flash, send_file
from flask_wtf import FlaskForm
from wtforms import FileField, SubmitField
from wtforms import StringField
from wtforms.validators import DataRequired
import ipfshttpclient
import os 
import time 
import torch
from transformers import pipeline
#from hugfacefunc import func
from transformers import pipeline
pipe = pipeline("text2text-generation", model="Shxck69/DimSum")


In [12]:
from importlib.metadata import version
version('wtforms')

'3.1.1'

In [ ]:
from io import BytesIO
app = Flask(__name__)
app.config['SECRET_KEY'] = 'your_secret_key'

# Function to save file to IPFS
#import os
#file_mapping={}
def save_to_ipfs(file):
    try:
        # Get the file name
        file_name = file.filename

        # Connect to the IPFS daemon
        client = ipfshttpclient.connect('/ip4/127.0.0.1/tcp/5001/http')

        # Save the file to a temporary location
        #temp_path = os.path.join('/tmp', file_name)
        #file.save(temp_path)
        # Open and read the file content
        with open(file_name, 'rb') as file:
            res = client.add(file)
            return res['Hash']
        # Add the file to IPFS
        #res = client.add(temp_path)

        # Remove the temporary file
        #os.remove(temp_path)
        #file_mapping[file_name]=res['Hash']
        return res['Hash']
    except Exception as e:
        print(f"Error saving to IPFS: {str(e)}")
        return None

# Function to retrieve file from IPFS
def retrieve_from_ipfs(ipfs_hash):
    try:
        client = ipfshttpclient.connect('/ip4/127.0.0.1/tcp/5001/http')
        file_data = client.cat(ipfs_hash)
        return file_data
    except Exception as e:
        print(f"Error retrieving from IPFS: {str(e)}")
        return None

# File Upload Form
class UploadForm(FlaskForm):
    file = FileField('File')
    submit = SubmitField('Upload')

# File upload route
@app.route('/upload', methods=['GET', 'POST'])
def upload():
    form = UploadForm()
    if form.validate_on_submit():
        # Save the file to IPFS
        ipfs_hash = save_to_ipfs(form.file.data)
        if ipfs_hash:
            flash(f'File uploaded to IPFS with hash: {ipfs_hash}', 'success')
        else:
            flash('Error uploading file to IPFS', 'error')
        return redirect(url_for('upload'))
    return render_template('upload.html', form=form)

# File download route
@app.route('/download',methods=['GET','POST'])
def download():
    # Retrieve the file from IPFS
    form=DownloadForm()
    if form.validate_on_submit():
        # Get the file name from the form
        file_name = form.file_name.data

        # Perform a lookup to retrieve the IPFS hash for the given file name
        ipfs_hash = file_name
        file_data = retrieve_from_ipfs(ipfs_hash)
        # Wrap the bytes data in a BytesIO object
        file_stream = BytesIO(file_data)

        if file_data:
            flash(f'File found! Download link generated for {file_name}', 'success')
            return send_file(file_stream, as_attachment=True, mimetype='application/octet-stream', download_name=f'{ipfs_hash}.txt')
        else:
            flash(f'File with name {file_name} not found', 'error')

        return redirect(url_for('download'))

    return render_template('download.html',form=form)

# Index route
@app.route('/')
def index():
    return render_template('index.html')

def narrative(ind,st):
    try:
        l = len(st) // 3
        h = str(st)
        p1 = h[ind:l]
        p2 = h[l:l+l]
        p3 = h[l+l:]
        #print(len(p1),len(p2),len(p3))
        s1 = pipe(p1, max_length=70, min_length=20, do_sample=False)
        s2  = pipe(p2,max_length=70,min_length=20,do_sample=False)
        s3  = pipe(p3,max_length=70,min_length=20,do_sample=False)
            #s4  = summarizer(p4,max_length=35,min_length=20,do_sample=False)
        return (s1[0]['generated_text']+s2[0]['generated_text']+s3[0]['generated_text'])
    except:
        narrative(ind+75,st)
   
    
# Function for HuggingFace processing (replace with actual processing logic)
def huggingface_processing(ipfs_hash):
    # Use a pipeline as a high-level helper
    #from transformers import pipeline
    #pipe = pipeline("text2text-generation", model="Shxck69/DimSum")
    f = retrieve_from_ipfs(ipfs_hash)
    st=str(f)
    if(len(st) > 3500):
        out = narrative(400,st)
    else:
        out=pipe(st,max_length=100,min_length=2,do_sample=False)
    #print(len(f))
    # Placeholder: You should implement the actual HuggingFace processing logic here
    # For example, you can simply return a modified version of the file data
    #return f'HuggingFace processed content for IPFS Hash: {ipfs_hash}'
    return out

# Process Form
class ProcessForm(FlaskForm):
    ipfs_hash = StringField('IPFS Hash', validators=[DataRequired()])
    submit = SubmitField('Process')

# Process route
@app.route('/process', methods=['GET', 'POST'])
def process():
    form = ProcessForm()
    if form.validate_on_submit():
        # Get the IPFS hash from the form
        ipfs_hash = form.ipfs_hash.data

        # Perform HuggingFace processing (replace with actual processing logic)
        processed_data = huggingface_processing(ipfs_hash)

        if processed_data:
            flash(f'HuggingFace processing complete for IPFS Hash: {ipfs_hash}', 'success')
            # You can display or redirect to a new page showing the processed data
            return render_template('processed_data.html', processed_data=processed_data)
        else:
            flash('Error during HuggingFace processing', 'error')
    return render_template('process.html', form=form)

# File Download Form
class DownloadForm(FlaskForm):
    file_name = StringField('File Name', validators=[DataRequired()])
    submit = SubmitField('Download')

# File download route
''''@app.route('/download', methods=['GET', 'POST'])
def download_file():
    form = DownloadForm()
    if form.validate_on_submit():
        # Get the file name from the form
        file_name = form.file_name.data

        # Perform a lookup to retrieve the IPFS hash for the given file name
        ipfs_hash = file_name

        if ipfs_hash:
            return redirect(url_for('download', ipfs_hash=ipfs_hash))
        else:
            flash(f'File with name {file_name} not found', 'error')

    return render_template('download.html', form=form)

@app.route('/download/<ipfs_hash>')
def download(ipfs_hash):
    file_data = retrieve_from_ipfs(ipfs_hash)
    if file_data:
        # Set the mimetype explicitly to avoid the ValueError
        return send_file(file_data, as_attachment=True, mimetype='application/octet-stream',download_name=f'{ipfs_hash}.file')
    else:
        flash('Error retrieving file from IPFS', 'error')
        return redirect(url_for('index'))

# File download route
@app.route('/download/<ipfs_hash>')
def download(ipfs_hash):
    # Simulate retrieving file data from IPFS based on the IPFS hash
    # Replace this with your actual logic to retrieve file data
    file_data = retrieve_from_ipfs(ipfs_hash)
    if file_data:
        return send_file(file_data, as_attachment=True, mimetype='application/octet-stream', download_name=f'{ipfs_hash}.file')
    else:
        flash('Error retrieving file from IPFS', 'error')
        return redirect(url_for('index'))


# Download link route
@app.route('/download_link', methods=['GET','POST'])
def download_link():
    form = DownloadForm()
    if form.validate_on_submit():
        # Get the file name from the form
        file_name = form.file_name.data

        # Perform a lookup to retrieve the IPFS hash for the given file name
        ipfs_hash = lookup_ipfs_hash_by_name(file_name)

        if ipfs_hash:
            flash(f'File found! Download link generated for {file_name}', 'success')
            return redirect(url_for('download', ipfs_hash=ipfs_hash))
        else:
            flash(f'File with name {file_name} not found', 'error')

    return redirect(url_for('download'))'''




if __name__ == '__main__':
    app.run(debug=False)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [09/Dec/2023 11:08:51] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [09/Dec/2023 11:08:51] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [09/Dec/2023 11:08:53] "GET /upload HTTP/1.1" 200 -
127.0.0.1 - - [09/Dec/2023 11:09:05] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [09/Dec/2023 11:09:12] "GET /upload HTTP/1.1" 200 -
127.0.0.1 - - [09/Dec/2023 11:09:25] "GET /process HTTP/1.1" 200 -
127.0.0.1 - - [09/Dec/2023 11:09:28] "GET /download HTTP/1.1" 200 -
127.0.0.1 - - [09/Dec/2023 11:09:34] "GET /upload HTTP/1.1" 200 -
127.0.0.1 - - [09/Dec/2023 11:09:42] "POST /upload HTTP/1.1" 302 -
127.0.0.1 - - [09/Dec/2023 11:09:42] "GET /upload HTTP/1.1" 200 -
127.0.0.1 - - [09/Dec/2023 11:09:46] "GET /upload HTTP/1.1" 200 -
127.0.0.1 - - [09/Dec/2023 11:09:49] "GET /process HTTP/1.1" 200 -
127.0.0.1 - - [09/Dec/2023 11:10:00] "POST /process HTTP/1.1" 200 -
127.0.0.1 - - [09/Dec/2023 11:10:47] "GET /process HTTP/1.1" 200 -
127.0.0.1 - - [09/Dec/2023 11